In [20]:
library(renv)
library(tidymodels)
library(tidyverse)
library(leaps)
library(caret)
library(kknn)

In [21]:
articles <- read_csv('data/OnlineNewsPopularity.csv')
head(data)

Rows: 39644 Columns: 61
── Column specification ─────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): url
dbl (60): timedelta, n_tokens_title, n_tokens_content, n_unique_tokens, n_no...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


                                                                            
1 function (..., list = character(), package = NULL, lib.loc = NULL,        
2     verbose = getOption("verbose"), envir = .GlobalEnv, overwrite = TRUE) 
3 {                                                                         
4     fileExt <- function(x) {                                              
5         db <- grepl("\\\\.[^.]+\\\\.(gz|bz2|xz)$", x)                     
6         ans <- sub(".*\\\\.", "", x)                                      

To lighten the compute load, going to drop columns that are more designed for natural language processing as well as `weekday_is_sunday` since the other 6 days are already present as attributes, meaning `weekday_is_sunday` will have no impact in the model.

In [22]:
drop_columns <- c('url', 'timedelta', 'LDA_00', 'LDA_01', 'LDA_02','LDA_03','LDA_04', 'is_weekend', 'weekday_is_sunday',
'global_subjectivity','global_sentiment_polarity','global_rate_positive_words','global_rate_negative_words',
'rate_positive_words','rate_negative_words','avg_positive_polarity','min_positive_polarity','max_positive_polarity',
'avg_negative_polarity','min_negative_polarity','max_negative_polarity','title_subjectivity','title_sentiment_polarity',
'abs_title_subjectivity','abs_title_sentiment_polarity','kw_min_min','kw_max_min',
'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg' )


articles <- articles[, !(names(articles) %in% drop_columns)]

In [23]:
set.seed(2024)

articles$ID <- 1:nrow(articles)

training_articles <- sample_n(articles, size = nrow(articles) * 0.70, replace = FALSE)

testing_articles <- anti_join(articles, training_articles, by = "ID")

head(training_articles, 3)
nrow(training_articles)

head(testing_articles, 3)
nrow(testing_articles)

n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,⋯,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,shares,ID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
9,549,0.4935305,1,0.6827586,8,7,2,0,4.633880,⋯,3500,2396.5,0,1,0,0,0,0,1500,21029
9,452,0.5193622,1,0.6000000,18,1,13,0,4.692478,⋯,803,803.0,0,1,0,0,0,0,1400,19872
7,1361,0.3924890,1,0.6248237,8,3,1,1,4.539309,⋯,4300,4300.0,1,0,0,0,0,0,1500,7802


[1] 27750

n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,⋯,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,shares,ID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
9,255,0.6047431,1,0.7919463,3,1,1,0,4.913725,⋯,0,0,1,0,0,0,0,0,711,2
9,211,0.5751295,1,0.6638655,3,1,1,0,4.393365,⋯,918,918,1,0,0,0,0,0,1500,3
9,285,0.7441860,1,0.8415301,4,2,0,21,4.343860,⋯,22800,11785,1,0,0,0,0,0,10000,14


[1] 11894

Going to fit an initial linear model using all attributes and ordinary least squares(OLS) to receive a baseline model

In [24]:
article_full_OLS <- lm(shares ~ ., data = training_articles[,-28])

tidy(article_full_OLS)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),4.928611e+03,6.568622e+02,7.50326474,6.411454e-14
n_tokens_title,9.308998e+01,3.263221e+01,2.85270220,4.338129e-03
n_tokens_content,2.824141e-01,2.510659e-01,1.12486023,2.606580e-01
n_unique_tokens,2.498136e+03,1.878005e+03,1.33020752,1.834608e-01
n_non_stop_words,-6.728982e+02,7.093854e+02,-0.94856499,3.428502e-01
n_non_stop_unique_tokens,-1.611476e+03,1.625585e+03,-0.99132088,3.215376e-01
num_hrefs,4.471285e+01,7.501829e+00,5.96026063,2.548771e-09
num_self_hrefs,-8.275621e+01,2.131307e+01,-3.88288605,1.034632e-04
num_imgs,1.562504e+01,1.036265e+01,1.50782226,1.316114e-01


In [25]:
articles_test_pred_full_OLS <- predict(article_full_OLS, newdata = testing_articles[, -28])
head(articles_test_pred_full_OLS)

1        2        3        4        5        6 
2067.922 2497.793 5570.802 4869.702 3230.057 1902.576

In [26]:
articles_RMSE_models <- tibble(
    Model = 'OLS Full Regression',
    RMSE = RMSE(
        articles_test_pred_full_OLS,
        testing_articles$shares
    )
)
articles_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,12079.27


Since we have multiple attributes, we wish to find the most significant attributes that contribute the most to the amount of shares an article receives. We will do so using forward selection by way of the `tidymodels` package.

In [27]:
articles_forward_sel <- regsubsets(
    x = shares ~ ., nvmax = 27,
    data = training_articles[, -28],
    method = 'forward' 
)

articles_forward_summary <- summary(articles_forward_sel)
articles_forward_summary

Subset selection object
Call: regsubsets.formula(x = shares ~ ., nvmax = 27, data = training_articles[, 
    -28], method = "forward")
26 Variables  (and intercept)
                              Forced in Forced out
n_tokens_title                    FALSE      FALSE
n_tokens_content                  FALSE      FALSE
n_unique_tokens                   FALSE      FALSE
n_non_stop_words                  FALSE      FALSE
n_non_stop_unique_tokens          FALSE      FALSE
num_hrefs                         FALSE      FALSE
num_self_hrefs                    FALSE      FALSE
num_imgs                          FALSE      FALSE
num_videos                        FALSE      FALSE
average_token_length              FALSE      FALSE
num_keywords                      FALSE      FALSE
data_channel_is_lifestyle         FALSE      FALSE
data_channel_is_entertainment     FALSE      FALSE
data_channel_is_bus               FALSE      FALSE
data_channel_is_socmed            FALSE      FALSE
data_channel_is_tec

In [28]:
articles_forward_summary_df <- tibble(
    n_input_variables = 1:26,
    RSQ = articles_forward_summary$rsq,
    RSS = articles_forward_summary$rss,
    ADJ.R2 = articles_forward_summary$adjr2,
    Cp = articles_forward_summary$cp,
    BIC = articles_forward_summary$bic,
)
articles_forward_summary_df

n_input_variables,RSQ,RSS,ADJ.R2,Cp,BIC
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.002570620,3.593809e+12,0.002534674,360.38289,-50.96456
2,0.004700896,3.586133e+12,0.004629155,302.35421,-100.06448
3,0.006233004,3.580613e+12,0.006125555,261.18122,-132.58321
4,0.007550722,3.575865e+12,0.007407641,226.04948,-159.17265
5,0.008335362,3.573038e+12,0.008156645,205.93926,-170.88975
6,0.009201145,3.569919e+12,0.008986865,183.54250,-184.89678
7,0.010528182,3.565137e+12,0.010278513,148.14818,-211.85794
8,0.012430877,3.558282e+12,0.012146080,96.53249,-255.03990
9,0.013564533,3.554197e+12,0.013244492,66.58740,-276.68214


Since we are doing predictive modelling, we will use Mallow's $C_p$ to assess the performance of the different predictive models, where the smaller the $C_p$ value is, the more precise the model relatively is. In this case, using 15 input variables has the smallest $C_p$ value.

The 15 input variables are:
```
n_tokens_title
n_tokens_content
num_hrefs
num_self_hrefs
average_token_length
data_channel_is_lifestyle
data_channel_is_entertainment
data_channel_is_bus
data_channel_is_socmed
data_channel_is_tech
data_channel_is_world
self_reference_min_shares
self_reference_max_shares
self_reference_avg_shares
weekday_is_wednesday

```

Now a reduced model is made using these 15 input variables.

In [29]:
articles_red_OLS <- lm(shares ~ n_tokens_title + n_tokens_content + num_hrefs + num_self_hrefs +
        average_token_length + data_channel_is_lifestyle + data_channel_is_entertainment + data_channel_is_bus + 
        data_channel_is_socmed +  data_channel_is_tech + data_channel_is_world + self_reference_min_shares + 
        self_reference_max_shares + self_reference_avg_sharess + weekday_is_wednesday,
        data = training_articles
    )

articles_test_pred_red_OLS <- predict(articles_red_OLS, newdata = testing_articles[, -28])

In [30]:
articles_RMSE_models <- rbind(
    articles_RMSE_models,
    tibble(
        Model = "OLS Reduced Regression",
        RMSE = RMSE(
            articles_test_pred_red_OLS,
            testing_articles$shares
        )
    )
)
articles_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,12079.27
OLS Reduced Regression,12081.84


Will also use a K-nearest neighbours model to predict shares using `tidymodels` and 5-fold cross validation to choose the apporiate amount of neighbours using minimum RMSPE.

In [32]:
articles_recipe <- recipe(shares ~ n_tokens_title + n_tokens_content + num_hrefs + num_self_hrefs +
        average_token_length + data_channel_is_lifestyle + data_channel_is_entertainment + data_channel_is_bus + 
        data_channel_is_socmed +  data_channel_is_tech + data_channel_is_world + self_reference_min_shares + 
        self_reference_max_shares + self_reference_avg_sharess + weekday_is_wednesday, data = training_articles) %>%
        step_scale(all_predictors()) %>%
        step_center(all_predictors())

articles_spec <- nearest_neighbor(weight_func = 'rectangular',
                                  neighbors = tune()) %>%
    set_engine('kknn') %>%
    set_mode('regression')

articles_vfold <- vfold_cv(training_articles, v = 5, strata = shares)



In [33]:
gridvals <- tibble(neighbors = seq(1, 30))

articles_multi <- workflow() |>
  add_recipe(articles_recipe) |>
  add_model(articles_spec) |>
  tune_grid(articles_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse") |>
  filter(mean == min(mean))

articles_k <- articles_multi |>
              pull(neighbors)
articles_multi

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
30,rmse,standard,11306.96,5,799.0212,Preprocessor1_Model30


We can compare the mean RMSE from K-Nearest Neighbours to the OLS models from earlier, and see that the K-Nearest Neighbours had a smaller RMSE, meaning it performs better in this scenario.